# Data Scientist

In [ ]:
from src.AriesDuetTokenExchanger import AriesDuetTokenExchanger

%autoawait
import time
import asyncio
import os
import nest_asyncio
from aries_cloudcontroller import AriesAgentController

In [ ]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

print(
    f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}"
)
scientist_controller = AriesAgentController(admin_url, api_key)

await scientist_controller.init_webhook_server(webhook_host, webhook_port)

In [ ]:
datascientist_exchanger = AriesDuetTokenExchanger(agent_controller=scientist_controller)

## Get Credential from OM Duet Authority Issuer 

Again you should produce this invite through the relevant entrypoint exposed by the OM Authority user interface.

Accepting a connection using the correct invite from the duet issuer will result in the scientists agent being issued a credential from the Duet Issuer which they can use to authenticate with the Data Owner.

Note: The process by which the Data Scientist gets a credential can be designed to provide context specific trust to the use case. They might need to fill out a form, get approval from an ethics committee or prove they are a member of an organisation. The credential acts as a digital attestation to the successful completion of the defined process made by a trustworthy entity.

In [ ]:
invite = {
    "@type":"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation",
    "@id":"ebf3560d-ddea-473f-9e4b-fec688ae1590",
    "label":"OpenMined Duet Authority",
    "recipientKeys":["3YwzJ2pfs75DxuuqvRRWCiPyqce68iA67W9Jb6dkQn6X"],
    "serviceEndpoint":"http://139.162.224.50:3020"
}

In [ ]:
connection_id = datascientist_exchanger.receive_invitation(invite)

## Configure Duet Server Challenge

This is a aries proof request that is made to every entity who attempts to make a connection with the Duet server. Before receiving the relevant token they must respond with a valid presentation to this request.

Note this challenge could be anything you want, designed to establish context specific trust dependant on the use case. For the challenge to be useful you must be confident that the correct entities will be able to get access to the credential needed to respond. 

In this example case they need a credential based on a schema with this ID `Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1`. Getting this credential is easy, entities simply need to connect to the Duet Issuer and will be automatically issued one. An example of a more realistic scenario could include application for research approval from an ethics committee.

In [ ]:
schema_id = "Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1"


revocation = False
exchange_tracing = False

# Enable this to ask for attributes to identity a user
# TODO - change restriction to schemaId or credentialId

req_attrs = [
    {"name": "domain", "restrictions": [{"schema_id": schema_id}]},
]

global indy_proof_request
indy_proof_request = {
    "name": "Proof of Scientist",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid": req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        #         f"0_{req_pred['name']}_GE_uuid":
        #         req_pred for req_pred in req_preds
    },
}

In [ ]:
datascientist_exchanger.configure_challenge(indy_proof_request)

## Join Duet Session

In this instance rather than launching a new Duet server, we are going to join the one created by the Data Owner. Again we pass in a AriesExchanger as the DuetTokenExchanger. This time instead of creating an invitation, the Duet instance requests a invitation be submitted as an input. 

Once this is done, the data scientists agent will establish a connection with the data owner and and authentication policies that were configured will be requested. If successful then the respective parties will communicate their Duet token identifiers over the authenticated DIDComm channel. This will be used to establish a Duet WebRTC connection across with PPML messages can be exchanged.

In [ ]:
import syft as sy

duet = sy.join_duet(credential_exchanger=datascientist_exchanger)

## Congratulations!

Once a Duet session has been created, you should be able to run any PPML flows that Duet / PySyft supports. This is left as an excercise for the interested. Do let me know if you run into any problems, but this has been designed so that the authentication layer is not tightly coupled to the Syft flows. Rather it act's as an entrypoint to a PPML flow.

In [ ]:
duet.torch